 # Competición Sales Prediction con keras


 ## importar librerias y datos

In [68]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.sequence import TimeseriesGenerator
# from sklearn.model_selection import train_test_split

In [69]:
item_categories = pd.read_csv('data/item_categories.csv')
items = pd.read_csv('data/items.csv')
sales_train = pd.read_csv('data/sales_train.csv')
sample_submission = pd.read_csv('data/sample_submission.csv')
shops = pd.read_csv('data/shops.csv')
test = pd.read_csv('data/test.csv')

datasets = {
    'item_categories': item_categories.columns.values,
    'items': items.columns.values,
    'sales_train': sales_train.columns.values,
    'sample_submission': sample_submission.columns.values,
    'shops': shops.columns.values,
    'test': test.columns.values
}

# Si cada dataset tiene menos de 6 columnas, completar con NaN.
for dataset in datasets:
    while len(datasets[dataset]) < 6:
        datasets[dataset] = np.append(datasets[dataset], '-')

pd.DataFrame(datasets)

,item_categories,items,sales_train,sample_submission,shops,test
0,item_category_name,item_name,date,ID,shop_name,ID
1,item_category_id,item_id,date_block_num,item_cnt_month,shop_id,shop_id
2,-,item_category_id,shop_id,-,-,item_id
3,-,-,item_id,-,-,-
4,-,-,item_price,-,-,-
5,-,-,item_cnt_day,-,-,-


In [70]:
ds = sales_train

ds.rename(columns={'item_cnt_day':'sales', 'date_block_num':'block'}, inplace=True)
v = ds.groupby(['block','shop_id','item_id']).sum()['sales'].reset_index()
w = ds.groupby(['block','shop_id','item_id']).mean()['item_price'].reset_index()
v.insert(v.shape[-1]-1, 'mean_monthly_price', w.item_price)
v = v.loc[v['sales'] != 0]
ds = v.sort_values(by=['item_id', 'shop_id', 'block'])


# ds.loc[ds['item_id']==33]

In [71]:
ds_pivoted = ds.pivot_table(index = ['shop_id','item_id'], values = ['mean_monthly_price', 'sales'], columns = ['block'], fill_value = 0, aggfunc='sum').reset_index()
ds_pivoted.head()

shop_id item_id mean_monthly_price                                       \
block                                  0      1    2    3    4    5    6    7   
0           0      30                0.0  265.0  0.0  0.0  0.0  0.0  0.0  0.0   
1           0      31                0.0  434.0  0.0  0.0  0.0  0.0  0.0  0.0   
2           0      32              221.0  221.0  0.0  0.0  0.0  0.0  0.0  0.0   
3           0      33              347.0  347.0  0.0  0.0  0.0  0.0  0.0  0.0   
4           0      35              247.0  247.0  0.0  0.0  0.0  0.0  0.0  0.0   

       ... sales                             
block  ...    24 25 26 27 28 29 30 31 32 33  
0      ...     0  0  0  0  0  0  0  0  0  0  
1      ...     0  0  0  0  0  0  0  0  0  0  
2      ...     0  0  0  0  0  0  0  0  0  0  
3      ...     0  0  0  0  0  0  0  0  0  0  
4      ...     0  0  0  0  0  0  0  0  0  0  

[5 rows x 70 columns]

In [72]:
shape_0 = ds.shape
for i in range(34):
    a = ds_pivoted.loc[ds_pivoted['sales'][i]==0]
    ds = pd.concat([ds, pd.DataFrame({'shop_id':a['shop_id'], 'item_id':a['item_id'], 'mean_monthly_price':a['mean_monthly_price'][i], 'block':i, 'sales':0})], ignore_index=True)

shape_1 = ds.shape
print(shape_0, shape_1)
ds = ds.sort_values(by=['item_id', 'shop_id', 'block'])
ds.head(40)

(1606541, 5) (14411682, 5)


,block,shop_id,item_id,mean_monthly_price,sales
1918541,0,54,0,0.0,0.0
2281918,1,54,0,0.0,0.0
2646808,2,54,0,0.0,0.0
3014977,3,54,0,0.0,0.0
3385565,4,54,0,0.0,0.0
3754115,5,54,0,0.0,0.0
4120662,6,54,0,0.0,0.0
4485986,7,54,0,0.0,0.0
4857327,8,54,0,0.0,0.0
5229869,9,54,0,0.0,0.0


In [73]:
# Transformar el tipo de datos de la columna 'shop_id' a string.
ids = ds['shop_id'].astype(str)
# Unir una columna con el valor de la columna 'shop_id' y la columna 'item_id'
ids = ids.str.cat(ds['item_id'].astype(str), sep='_')
ids.unique().shape

(423873,)

In [133]:
# X_train = ds.values[:,:-1]
# X_valid = ds.values[:,1:]
generator = TimeseriesGenerator(ds.values[:], ds.values, length=33, batch_size=81, stride=34, shuffle=False)

In [131]:
ds.shape[0]/(34*81)

5233.0

 ## Definición de modelo

In [134]:
def build_model(lr):

  model = keras.Sequential([
    keras.layers.LSTM(units=32, batch_input_shape=(81, 33, 5), stateful=True),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(1, activation='relu')
  ])

  model.compile(loss='mse',
                optimizer=Adam(learning_rate=lr),
                metrics=['accuracy'])

  return model

 ## Entrenamiento del modelo


### Instaciacion del modelo.

In [135]:
keras.utils.set_random_seed(0)
model = build_model(0.01)

### entrenamiento normal

In [137]:
history = model.fit(generator, verbose=2, epochs=25, steps_per_epoch=len(generator))

Epoch 1/25


KeyboardInterrupt: 

 ## Evaluar con data 100% accuracy
 para evitar tener que estar subiendo constantemente la submission a kaggle, lo comparamos con el resultado final de la competición que se encuentra en un repo en github.

In [ ]:
y_test = pd.read_csv('data/submission_100accuracy.csv')['Survived']

model.evaluate(test, y_test)

 ## Predicciones

In [ ]:
preds = model.predict(test)
preds[preds > 0.5] = 1
preds[preds <= 0.5] = 0

 ## Exportar predicciones

In [ ]:
prediction = pd.DataFrame({'PassengerId': test_data.PassengerId.values, 'Survived': preds.ravel().astype('int64')})
prediction.to_csv('data/results.csv', index=False)

In [ ]:
prediction.head(5)